In [26]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
# from awsglue.job import Job

## Additional Lib
from pyspark.sql.functions import *
from awsglue.dynamicframe import DynamicFrame
##


# ## @params: [JOB_NAME]
# args = getResolvedOptions(sys.argv, ['JOB_NAME'])

# sc = SparkContext()
glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session
# job = Job(glueContext)
# job.init(args['JOB_NAME'], args)
## @type: DataSource
## @args: [database = "default", table_name = "donuts_xml", transformation_ctx = "datasource0"]
## @return: datasource0
## @inputs: []
datasource0 = glueContext.create_dynamic_frame.from_catalog(database = "default",
                                                            table_name = "donuts_xml",
                                                            transformation_ctx = "datasource0")


In [2]:
s3_path = "s3://aws-glue-elif-test/donuts.xml"
d0 = glueContext.create_dynamic_frame.from_options("s3",
                                                   {'paths': [s3_path] },
                                                   format="xml",
                                                   format_options={"rowTag":"item"},transformation_ctx = "d0")

In [23]:
spark_df = spark.read.format("com.databricks.spark.xml").options(rowTag="item").options(nullValue="").load(s3_path)

In [24]:
spark_df.printSchema()

root
 |-- _id: long (nullable = true)
 |-- _type: string (nullable = true)
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _id: long (nullable = true)
 |-- fillings: struct (nullable = true)
 |    |-- filling: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _id: long (nullable = true)
 |    |    |    |-- addcost: double (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _id: long (nullable = true)

In [28]:
spark_df.withColumn('batters', explode(col('batters')['batter'])).show()

+---+------+-------------------+--------------------+----------+----+--------------------+
|_id| _type|            batters|            fillings|      name| ppu|             topping|
+---+------+-------------------+--------------------+----------+----+--------------------+
|  1| donut|     [Regular,1001]|                null|      Cake|0.55|[[None,5001], [Gl...|
|  1| donut|   [Chocolate,1002]|                null|      Cake|0.55|[[None,5001], [Gl...|
|  1| donut|   [Blueberry,1003]|                null|      Cake|0.55|[[None,5001], [Gl...|
|  1| donut|[Devil's Food,1003]|                null|      Cake|0.55|[[None,5001], [Gl...|
|  2| donut|     [Regular,1001]|                null|    Raised|0.55|[[None,5001], [Gl...|
|  3| donut|     [Regular,1001]|                null|Buttermilk|0.55|                null|
|  3| donut|   [Chocolate,1002]|                null|Buttermilk|0.55|                null|
|  4|   bar|     [Regular,1001]|[WrappedArray([70...|       Bar|0.75|[[Chocolate,5003]...|

In [29]:
spark_df.schema.names

['_id', '_type', 'batters', 'fillings', 'name', 'ppu', 'topping']

In [30]:
spark_df.dtypes[spark_df.schema.names.index('batters')]

('batters', 'struct<batter:array<struct<_VALUE:string,_id:bigint>>>')

In [3]:
s3_staging_path = "s3://aws-glue-elif-test/temp"

In [6]:
r0 = Relationalize.apply(frame = d0, 
                         staging_path= s3_staging_path, 
                         name='root',
                         options=None, 
                         transformation_ctx = "r0")

In [7]:
r0.keys()

[u'root_topping', u'root_batters.batter.array', u'root', u'root_fillings.filling']

In [9]:
r0.select('root_fillings.filling').toDF().show()

+---+-----+------------------------+-----------------------------------+-------------------------+--------------------------------+
| id|index|fillings.filling.val._id|fillings.filling.val.addcost.double|fillings.filling.val.name|fillings.filling.val.addcost.int|
+---+-----+------------------------+-----------------------------------+-------------------------+--------------------------------+
|  1|    0|                    7001|                                0.0|                     None|                            null|
|  1|    1|                    7002|                               0.25|                  Custard|                            null|
|  1|    2|                    7003|                               0.25|            Whipped Cream|                            null|
|  2|    0|                    7002|                               null|                  Custard|                               0|
|  2|    1|                    7003|                               null|    

In [10]:
r0.select('root').toDF().show()

+---+------+----------------------------+-------------------------+--------------------+----------+----+-------+----------------+
|_id| _type|batters.batter.struct._VALUE|batters.batter.struct._id|batters.batter.array|      name| ppu|topping|fillings.filling|
+---+------+----------------------------+-------------------------+--------------------+----------+----+-------+----------------+
|  1| donut|                        null|                     null|                   1|      Cake|0.55|      1|            null|
|  2| donut|                     Regular|                     1001|                null|    Raised|0.55|      2|            null|
|  3| donut|                        null|                     null|                   2|Buttermilk|0.55|   null|            null|
|  4|   bar|                     Regular|                     1001|                null|       Bar|0.75|      3|               1|
|  5| twist|                     Regular|                     1001|                null|  

In [11]:
r0.select('root_batters.batter.array').toDF().show()

+---+-----+-------------------------------+----------------------------+
| id|index|batters.batter.array.val._VALUE|batters.batter.array.val._id|
+---+-----+-------------------------------+----------------------------+
|  1|    0|                        Regular|                        1001|
|  1|    1|                      Chocolate|                        1002|
|  1|    2|                      Blueberry|                        1003|
|  1|    3|                   Devil's Food|                        1003|
|  2|    0|                        Regular|                        1001|
|  2|    1|                      Chocolate|                        1002|
+---+-----+-------------------------------+----------------------------+

In [19]:
un0 = UnnestFrame.apply(frame = d0, transformation_ctx = "un0")

In [20]:
un0.toDF().printSchema()

root
 |-- _id: integer (nullable = true)
 |-- _type: string (nullable = true)
 |-- batters.batter._VALUE: string (nullable = true)
 |-- batters.batter._id: integer (nullable = true)
 |-- fillings.filling: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- fillings.filling.val._id: integer (nullable = true)
 |    |    |-- fillings.filling.val.addcost: struct (nullable = true)
 |    |    |    |-- double: double (nullable = true)
 |    |    |    |-- int: integer (nullable = true)
 |    |    |-- fillings.filling.val.name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- topping.val._VALUE: string (nullable = true)
 |    |    |-- topping.val._id: integer (nullable = true)
 |-- batters.batter: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- batters.batter.val._VALUE: strin

In [21]:
un0.toDF().show()

+---+------+---------------------+------------------+--------------------+----------+----+--------------------+--------------------+
|_id| _type|batters.batter._VALUE|batters.batter._id|    fillings.filling|      name| ppu|             topping|      batters.batter|
+---+------+---------------------+------------------+--------------------+----------+----+--------------------+--------------------+
|  1| donut|                 null|              null|                null|      Cake|0.55|[[None,5001], [Gl...|[[Regular,1001], ...|
|  2| donut|              Regular|              1001|                null|    Raised|0.55|[[None,5001], [Gl...|                null|
|  3| donut|                 null|              null|                null|Buttermilk|0.55|                null|[[Regular,1001], ...|
|  4|   bar|              Regular|              1001|[[7001,[0.0,null]...|       Bar|0.75|[[Chocolate,5003]...|                null|
|  5| twist|              Regular|              1001|                